In [1]:
%load_ext autoreload
%autoreload 2

import joblib
from joblib import Parallel, delayed
import multiprocessing

from tqdm import tqdm, tqdm_notebook

import pandas as pd
import numpy as np
import gzip
import json
import math
from datetime import datetime
import seaborn as sns
import matplotlib.dates as mdates
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display
import pickle

import sys
sys.path.append('..')
from helper.rl_framework import *
from helper.orderbook_container import OrderbookContainer
from helper.manage_orderbooks import *
from helper.orderbook_trader import *
from helper.evaluation import evaluate, plot_evaluation_costs
from helper.general_helpers import add_features_to_orderbooks, load_and_preprocess_historyfiles

from agents.RL_Agent_Base import RLAgent_Base
from agents.BatchTree_Agent import RLAgent_BatchTree
from agents.QTable_Agent import QTable_Agent
from Runs.train_agents import trainer_BatchTree, trainer_QTable

num_cores = multiprocessing.cpu_count()

In [2]:
data_may = pickle.load( open( '../cached_windows_60mins_V200/obs_2017-05_USDT_BTC_maxVol200.p', "rb" ) )

In [ ]:
print("data_mini")
data = pickle.load( open( "cached_windows/tradingwindows_1611_USTD_BTC_20.p", "rb" ) )
for window in data:
    for ob in window:
        ob.features = {}

In [ ]:
T=4
P=15
V=70000
consume='cash'
print("T: {}, P: {}, V: {}, consume: '{}'".format(T, P, V, consume))

# actions = np.linspace(-0.4, 1.0, num=15)
actions = range(-4,11)

print("Trading windows: {} (each one: {} minutes)".format(len(data), len(data[0])))

In [ ]:
### QTable Agent
agent = trainer_QTable(orderbooks=data[:24], V=V, T=T, consume=consume, actions=[round(a, 2) for a in actions],
                    limit_base='currAsk', vol_intervals=4,
                    period_length=P, agent_name='QTable_1611-1704_T4_I8_VolTime_Masterbookask',
                    state_variables=['volume', 'time'], mode='backward')
agent.save(path="trainedAgents/longterm_1611_1704_simulate_preceeding_trades_test", overwrite=True)

In [ ]:
agent.heatmap_Q()

In [ ]:
agent.heatmap_Q()
agent.save(path="trainedAgents/longterm_1611_1704_simulate_preceeding_trades_test", overwrite=False)

In [ ]:
agent.heatmap_Q()
# agent.save(path="trainedAgents/longterm_1611_1704_simulate_preceeding_trades_test", overwrite=False)

### Evaluate

In [3]:
import os
folder = 'trainedAgents/longterm_1611_1704_simulate_preceeding_trades_test'
folder = 'trainedAgents/longterm_BT'
agent_files = sorted([(f, folder) for f in os.listdir(folder) if f.endswith('.json')])

# agent_files.append( ('QTable_1611-1704_T4_I8_VolTime.json', 'trainedAgents/longterm_1611_1704_simulate_preceeding_trades_test') )
agent_files = [('BT_Agent_samples53184.json', 'trainedAgents/longterm_BT')]
print(agent_files)

[('BT_Agent_samples53184.json', 'trainedAgents/longterm_BT')]


In [4]:
agent_collection = {}
try:
    agent_collection
except NameError:
    agent_collection = {}
    
for elem, folder in agent_files:
    name = elem[:-5]
    print(name)
    agent_collection[name] = RLAgent_Base.load(agent_name=name, path=folder, ignore_samples=True)
    print(agent_collection[name])

BT_Agent_samples53184
RL-Type: <class 'agents.BatchTree_Agent.RLAgent_BatchTree'>, Name: 'BT_Agent_samples53184', state_variables: '['volume', 'time', 'level2data']'


In [ ]:
# baseline = 'test_Dec24_VolTime'
baseline = list(agent_collection.keys())[0]
costs, slippage = evaluate(
    testdata=data_may,
    agents=agent_collection,
    baseline=baseline,
#    evaluate_actions=[2, 4],
    verbose=False
)
slippage.to_csv('slippage_may_BT.csv')   # slippage_apr_manyVars3Bins_simulatedTrades, fixedMarketVar
plot_evaluation_costs(slippage, hline="2", showfliers=False)

Start parallel evalutions of 1 strategies over 736 tradingperiods. (num_cores=24)
eval


  0%|          | 0/1 [00:00<?, ?it/s]

eval


100%|██████████| 1/1 [00:13<00:00, 13.46s/it]


eval


...

100%|██████████| 1/1 [00:03<00:00,  3.12s/it]


eval


  0%|          | 0/1 [00:00<?, ?it/s]

eval


100%|██████████| 1/1 [00:08<00:00,  8.50s/it]


In [ ]:
display(slippage.mean())
plot_evaluation_costs(slippage, hline="2", showfliers=False)